In [674]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [675]:
from google.colab import drive, files
drive.mount('/content/drive')
rootpath = '/content/drive/Shareddrives/KW phase1 repo'
import sys
sys.path.append(rootpath+'/package')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [676]:
import cvxpy as cp
from utils.env import env
from model.socp import calculateSOCP, setSOCPEnv, SOCP
from model.pathlossmodel import PathLossModel, NoiseMaker
from service.measurerss import measureRSS

In [677]:
div = 100
pick = 3
model="VAE"

### Read final anomaly detection results

In [678]:
import pickle as dill
_UAVsSamples = {}
iSample = 0
dataFolder = "/1125_v2"
with open(rootpath+"/datas/anomalySOCPResult"+dataFolder+"/UAVsSamples_pick{}div{}_300.npy".format(pick,div), 'rb') as f:
  while True:
    if iSample >= env.LOAD_SAMPLES_FOR_AE:
       break
    try:
      _UAVsSamples[iSample] = dill.load(f)
      iSample += 1
    except EOFError:
      break
UAVsSamples = _UAVsSamples
print(UAVsSamples)

{0: {0: <model.uav.UAV object at 0x7fafe4163b10>, 1: <model.uav.UAV object at 0x7fafe4163f90>, 2: <model.uav.UAV object at 0x7fafe41630d0>, 3: <model.uav.UAV object at 0x7fafe4163c10>, 4: <model.uav.UAV object at 0x7fafe41635d0>, 5: <model.uav.UAV object at 0x7fafe4163410>, 6: <model.uav.UAV object at 0x7fafe41632d0>, 7: <model.uav.UAV object at 0x7fafe4163550>, 8: <model.uav.UAV object at 0x7fafe4163ed0>, 9: <model.uav.UAV object at 0x7fafe4163490>, 10: <model.uav.UAV object at 0x7fafe4163b50>, 11: <model.uav.UAV object at 0x7fafe4163e50>, 12: <model.uav.UAV object at 0x7fafe4163d50>, 13: <model.uav.UAV object at 0x7fafe4163310>, 14: <model.uav.UAV object at 0x7fafe4163f50>, 15: <model.uav.UAV object at 0x7fafe4163390>}, 1: {0: <model.uav.UAV object at 0x7fafe41631d0>, 1: <model.uav.UAV object at 0x7fafe41633d0>, 2: <model.uav.UAV object at 0x7fafe4163f10>, 3: <model.uav.UAV object at 0x7fafe4163990>, 4: <model.uav.UAV object at 0x7fafe4163210>, 5: <model.uav.UAV object at 0x7fafe4163

In [679]:
pathLossModel = PathLossModel()
pathLossModel.noiseMaker = NoiseMaker(mean = env.noiseMean, deviation = env.noiseDeviation)

In [680]:
Samples = len(UAVsSamples)
print("Samples : ", Samples)

Samples :  100


In [681]:
OriginAnchors = {}
OriginTargets = {}
for iSample in UAVsSamples:
  originAnchorsOfSample = []
  originTargetsOfSample = []
  for iUAVOfSample in UAVsSamples[iSample]:
    if UAVsSamples[iSample][iUAVOfSample].observedPosition == None:
      originTargetsOfSample.append(UAVsSamples[iSample][iUAVOfSample])
    else:
      originAnchorsOfSample.append(UAVsSamples[iSample][iUAVOfSample])
  OriginAnchors[iSample] = originAnchorsOfSample
  OriginTargets[iSample] = originTargetsOfSample

In [682]:
anomalyUAVsSample = {}
for iSample in UAVsSamples:
  anomalyUAV = []
  for IDOfUAV in UAVsSamples[iSample]:
    uav = UAVsSamples[iSample][IDOfUAV]
    if not uav.observedPosition == None:
      if not uav.observedPosition == uav.realPosition:
        anomalyUAV.append(IDOfUAV)
  anomalyUAVsSample[iSample] = anomalyUAV
anomalyUAVs = [anomalyUAVsSample]

In [683]:
OriginAnchors = OriginAnchors
OriginTargets = OriginTargets

In [684]:
import pickle
f = open(rootpath+"/datas/datas_forAnomalyDetection/BestAnchorsSOCP/1203_100sample_{}anomaly_{}div_SelectedAnchors_VAE_v2.pkl".format(pick,div), "rb")
goodUAVsSample = pickle.load(f)
f.close()

In [685]:
SelectedAnchors = {}
SelectedTargets = {}
for iSample in goodUAVsSample:
  selectedAnchorsOfSample = []
  selectedTargetsOfSample = []
  for IDOfUAV in UAVsSamples[iSample]:
    if IDOfUAV in goodUAVsSample[iSample]:
      selectedAnchorsOfSample.append(UAVsSamples[iSample][IDOfUAV])
    else:
      selectedTargetsOfSample.append(UAVsSamples[iSample][IDOfUAV])
  SelectedAnchors[iSample] = selectedAnchorsOfSample
  SelectedTargets[iSample] = selectedTargetsOfSample

In [686]:
SelectedAnchors = SelectedAnchors
SelectedTargets = SelectedTargets

### Calculate SOC-SDP

In [687]:
EstimatedPositionBeforeAnomalyDetection = {}
for Sample in range(Samples):
  print("#{:3d}".format(Sample))
  socp = SOCP()
  socp.var_t = cp.Variable(1)
  socp.var_p = [cp.Variable(env.DIMENSION) for i in range(len(OriginTargets[Sample]))]
  
  socp.buildConstraint(OriginTargets[Sample], OriginAnchors[Sample], pathLossModel)
  socp.doSolve(log=False)
  targetPositions = [socp.var_p[i].value for i in range(len(OriginTargets[Sample]))]
  EstimatedPositionBeforeAnomalyDetection[Sample] = targetPositions

#  0
#  1
#  2
#  3
#  4
#  5
#  6
#  7
#  8
#  9
# 10
# 11
# 12
# 13
# 14
# 15
# 16
# 17
# 18
# 19
# 20
# 21
# 22
# 23
# 24
# 25
# 26
# 27
# 28
# 29
# 30
# 31
# 32
# 33
# 34
# 35
# 36
# 37
# 38
# 39
# 40
# 41
# 42
# 43
# 44
# 45
# 46
# 47
# 48
# 49
# 50
# 51
# 52
# 53
# 54
# 55
# 56
# 57
# 58
# 59
# 60
# 61
# 62
# 63
# 64
# 65
# 66
# 67
# 68
# 69
# 70
# 71
# 72
# 73
# 74
# 75
# 76
# 77
# 78
# 79
# 80
# 81
# 82
# 83
# 84
# 85
# 86
# 87
# 88
# 89
# 90
# 91
# 92
# 93
# 94
# 95
# 96
# 97
# 98
# 99


In [688]:
EstimatedPositionAfterAnomalyDetection = {}
for Sample in range(Samples):
  print("#{:3d}".format(Sample))
  socp = SOCP()
  socp.var_t = cp.Variable(1)
  socp.var_p = [cp.Variable(env.DIMENSION) for i in range(len(SelectedTargets[Sample]))]
  
  socp.buildConstraint(SelectedTargets[Sample], SelectedAnchors[Sample], pathLossModel)
  socp.doSolve(log=False)
  targetPositions = [socp.var_p[i].value for i in range(len(SelectedTargets[Sample]))]
  EstimatedPositionAfterAnomalyDetection[Sample] = targetPositions

#  0
#  1
#  2
#  3
#  4
#  5
#  6
#  7
#  8
#  9
# 10
# 11
# 12
# 13
# 14
# 15
# 16
# 17
# 18
# 19
# 20
# 21
# 22
# 23
# 24
# 25
# 26
# 27
# 28
# 29
# 30
# 31
# 32
# 33
# 34
# 35
# 36
# 37
# 38
# 39
# 40
# 41
# 42
# 43
# 44
# 45
# 46
# 47
# 48
# 49
# 50
# 51
# 52
# 53
# 54
# 55
# 56
# 57
# 58
# 59
# 60
# 61
# 62
# 63
# 64
# 65
# 66
# 67
# 68
# 69
# 70
# 71
# 72
# 73
# 74
# 75
# 76
# 77
# 78
# 79
# 80
# 81
# 82
# 83
# 84
# 85
# 86
# 87
# 88
# 89
# 90
# 91
# 92
# 93
# 94
# 95
# 96
# 97
# 98
# 99


### Plot SOC-SDP result and localization RMSE

In [689]:
# import matplotlib.pyplot as plt

# fig = plt.figure(figsize=(10,10))
# ax = plt.axes(projection='3d')
# ax.set_xlim([0, 1500])
# ax.set_ylim([0, 1500])
# ax.set_zlim([0, 1500])
# ax.set_xlabel('x-axis (m)', fontsize=20)
# ax.set_ylabel('y-axis (m)', fontsize=20)
# ax.set_zlabel('z-axis (m)', fontsize=20)
# plt.grid()

# scatters = []
# scattersLegends = []

# sample = 45

# for originAnchor in OriginAnchors[sample]:
#   p = originAnchor.realPosition
#   if originAnchor.realPosition == originAnchor.observedPosition:
#     scatter1 = ax.scatter(p[0], p[1], p[2], c='#FF0000', marker='*', s=64, alpha=1)
#   else:
#     scatter_anomaly = ax.scatter(p[0], p[1], p[2], c='#000000', marker='^', s=64, alpha=1)
#     scatter_anomalyPosition = ax.scatter(originAnchor.observedPosition[0], originAnchor.observedPosition[1], originAnchor.observedPosition[2], c='orange', marker='^', s=64, alpha=1)
# scatters.append(scatter1)
# scattersLegends.append("Normal Anchor UAVs")
# scatters.append(scatter_anomaly)
# scattersLegends.append("Anomaly Anchor UAVs (True Position)")
# scatters.append(scatter_anomalyPosition)
# scattersLegends.append("Anomaly Anchor UAVs (Observed Position)")

# for originTarget in OriginTargets[sample]:
#   p = originTarget.realPosition
#   scatter2 = ax.scatter(p[0], p[1], p[2], c='#0000FF', marker='x', s=64, alpha=1)
# scatters.append(scatter2)
# scattersLegends.append("Target UAVs (without GPS)")

# for i, estimatedPosition in enumerate(EstimatedPositionBeforeAnomalyDetection[sample]):
#   p = estimatedPosition
#   scatter3 = ax.scatter(p[0], p[1], p[2], c='#00FF00', marker='o', s=64, alpha=1)
# scatters.append(scatter3)
# scattersLegends.append("Estimated Positions of Target UAVs")

# ax.view_init(20, -30)
# print('ax.azim {}'.format(ax.azim))
# print('ax.elev {}'.format(ax.elev))

# plt.legend(scatters, scattersLegends, loc='upper left', shadow=True, prop={'size': 16}, ncol=1)
# this_fig = plt.gcf()
# this_fig.savefig('localization_result_beforeDetection{}_{}_{}.eps'.format(div, pick, model), bbox_inches='tight')
# # files.download('localization_result_beforeDetection{}_{}_{}.eps'.format(div, pick, model))
# # this_fig.savefig('localization_result_beforeDetection{}_{}_{}.png'.format(div, pick, model), bbox_inches='tight')
# # files.download('localization_result_beforeDetection{}_{}_{}.png'.format(div, pick, model))

In [690]:
# import matplotlib.pyplot as plt

# fig = plt.figure(figsize=(10,10))
# ax = plt.axes(projection='3d')
# ax.set_xlim([0, 1200])
# ax.set_ylim([0, 1200])
# ax.set_zlim([0, 1200])
# ax.set_xlabel('x-axis (m)', fontsize=20)
# ax.set_ylabel('y-axis (m)', fontsize=20)
# ax.set_zlabel('z-axis (m)', fontsize=20)
# plt.grid()

# scatters = []
# scattersLegends = []

# sample = 45

# for originAnchor in SelectedAnchors[sample]:
#   p = originAnchor.realPosition
#   if originAnchor.realPosition == originAnchor.observedPosition:
#     scatter1 = ax.scatter(p[0], p[1], p[2], c='#FF0000', marker='*', s=64, alpha=1)
#   else:
#     scatter_anomaly = ax.scatter(p[0], p[1], p[2], c='#000000', marker='^', s=64, alpha=1)
#     scatters.append(scatter_anomaly)
#     scattersLegends.append("Anomaly Anchor UAVs")
# scatters.append(scatter1)
# scattersLegends.append("Selected Anchor UAVs")

# for originTarget in SelectedTargets[sample]:
#   p = originTarget.realPosition
#   if originTarget.observedPosition == None:
#     scatter2 = ax.scatter(p[0], p[1], p[2], c='#0000FF', marker='x', s=64, alpha=1)
#   else:
#     scatter_anomaly = ax.scatter(p[0], p[1], p[2], c='#000000', marker='x', s=64, alpha=1)
#     scatters.append(scatter_anomaly)
#     scattersLegends.append("Target UAVs (with Anomalous GPS)")
# scatters.append(scatter2)
# scattersLegends.append("Target UAVs (without GPS)")

# for i, estimatedPosition in enumerate(EstimatedPositionAfterAnomalyDetection[sample]):
#   p_bar = SelectedTargets[sample][i].realPosition
#   p = estimatedPosition
#   # plt.plot([p_bar[0], p[0]], [p_bar[1], p[1]], [p_bar[2], p[2]], c='#000000', linewidth=3.0)
#   scatter3 = ax.scatter(p[0], p[1], p[2], c='#00FF00', marker='o', s=64, alpha=1)
# scatters.append(scatter3)
# scattersLegends.append("Estimated Positions of Target UAVs")
# ax.view_init(20, -30)

# plt.legend(scatters, scattersLegends, loc='upper left', shadow=True, prop={'size': 16}, ncol=1)
# this_fig = plt.gcf()
# this_fig.savefig('localization_result_afterDetection{}_{}_{}.eps'.format(div, pick, model), bbox_inches='tight')
# # files.download('localization_result_afterDetection{}_{}_{}.eps'.format(div, pick, model))
# # this_fig.savefig('localization_result_afterDetection{}_{}_{}.png'.format(div, pick, model), bbox_inches='tight')
# # files.download('localization_result_afterDetection{}_{}_{}.png'.format(div, pick, model))


In [691]:
import numpy as np
MSE = lambda x,y: np.mean(np.square(np.array(x)-np.array(y)))

In [692]:
Before_Total_sumMSE = []
Before_Total_sumRMSE = []
Before_Each_sumMSE = []
Before_Each_sumRMSE = []
for Sample in range(Samples):
  origin_Nngps = len(OriginTargets[Sample])
  origin_Ngps = len(OriginAnchors[Sample])
  sumMSE = []
  sumRMSE = []
  for i in range(origin_Nngps):
    trueP = OriginTargets[Sample][i].realPosition
    estimatedP = EstimatedPositionBeforeAnomalyDetection[Sample][i]
    mse = MSE(trueP, estimatedP)
    rmse = np.sqrt(mse)
    # print("Targets: ", trueP, estimatedP, mse, rmse)
    sumMSE.append(mse)
    sumRMSE.append(rmse)
  for i in range(origin_Ngps):
    trueP = OriginAnchors[Sample][i].realPosition
    estimatedP = OriginAnchors[Sample][i].observedPosition
    if not trueP == estimatedP:
      mse = MSE(trueP, estimatedP)
      rmse = np.sqrt(mse)
      # print("Anchors: ", trueP, estimatedP, mse, rmse)
      sumMSE.append(mse)
      sumRMSE.append(rmse)
  Before_Total_sumMSE.append(sum(sumMSE))
  Before_Each_sumMSE.append(sumMSE)
  Before_Total_sumRMSE.append(sum(sumRMSE))
  Before_Each_sumRMSE.append(sumRMSE)

avg_sumMSE = np.mean(Before_Total_sumMSE)/env.N
avg_sumRMSE = np.mean(Before_Total_sumRMSE)/env.N
print("Average MSE: {}, RMSE: {}.".format(avg_sumMSE, avg_sumRMSE))

import os
try:
  os.makedirs(rootpath+'/datas/localizationResult')
except OSError as error:
  print(error)

# np.save(rootpath+'/datas/localizationResult/before_localization_avgMSE{}_{}_{}'.format(div, pick, model), avg_sumMSE)
# np.save(rootpath+'/datas/localizationResult/before_localization_avgRMSE{}_{}_{}'.format(div, pick, model), avg_sumRMSE)
f = open(rootpath+'/datas/localizationResult/1213_before_localization_avgRMSE{}_{}_{}_v2.pkl'.format(div, pick, model), "wb")
pickle.dump(avg_sumRMSE, f)
f.close()

# np.save(rootpath+'/datas/localizationResult/before_localization_MSE{}_{}_{}'.format(div, pick, model), Before_Each_sumMSE)
# np.save(rootpath+'/datas/localizationResult/before_localization_RMSE{}_{}_{}'.format(div, pick, model), Before_Each_sumRMSE)

# f = open(rootpath+'/datas/localizationResult/before_localization_MSE{}_{}_{}.pkl'.format(div, pick, model), "wb")
# pickle.dump(Before_Each_sumMSE, f)
# f.close()

f = open(rootpath+'/datas/localizationResult/1213_before_localization_RMSE{}_{}_{}_v2.pkl'.format(div, pick, model), "wb")
pickle.dump(Before_Each_sumRMSE, f)
f.close()

Average MSE: 466364.94818082795, RMSE: 518.8396159554849.
[Errno 17] File exists: '/content/drive/Shareddrives/KW phase1 repo/datas/localizationResult'


In [693]:
# print(Before_Each_sumRMSE)
# # print(np.array(Before_Each_sumMSE).flatten().tolist())
# Before_sumRMSE = np.concatenate(([], np.array(Before_Each_sumRMSE).flatten().tolist()),axis=0)
# # Before_sumRMSE = [np.log(y) for y in Before_sumRMSE]
# print(Before_sumRMSE)
# Before_RMSEHist, Before_bin_edges = np.histogram(Before_sumRMSE, bins=1000, range=(0, 1.6*1e3), density=True)
# normalizeBefore_RMSEHist = Before_RMSEHist*np.diff(Before_bin_edges)

# Before_cdf=np.cumsum(normalizeBefore_RMSEHist)

# import matplotlib.pyplot as plt
# fig, ax1 = plt.subplots(1,1, figsize = (8,8))
# ax1.plot(Before_bin_edges[:-1], Before_cdf, label = 'Localization Error cdf', linewidth=3.0)
# # ax1.hist(Before_sumMSE, bins = 1000, label = 'Localization Error cdf', range=(0, 1.0*1e6), alpha = 0.5, cumulative=True, density=True)
# ax1.legend()
# ax1.set_xlabel('Localization Error')

In [694]:
After_Total_sumMSE = []
After_Total_sumRMSE = []
After_Each_sumMSE = []
After_Each_sumRMSE = []
for Sample in range(Samples):
  new_Nngps = len(SelectedTargets[Sample])
  new_Ngps = len(SelectedAnchors[Sample])
  sumMSE = []
  sumRMSE = []
  for i in range(new_Nngps):
    trueP = SelectedTargets[Sample][i].realPosition
    try:
      estimatedP = EstimatedPositionAfterAnomalyDetection[Sample][i]
    except:
      pass
    mse = MSE(trueP, estimatedP)
    rmse = np.sqrt(mse)
    # print("Targets: ", trueP, estimatedP, mse, rmse)
    sumMSE.append(mse)
    sumRMSE.append(rmse)
  for i in range(new_Ngps):
    trueP = SelectedAnchors[Sample][i].realPosition
    estimatedP = SelectedAnchors[Sample][i].observedPosition
    if not trueP == estimatedP:
      mse = MSE(trueP, estimatedP)
      rmse = np.sqrt(mse)
      # print("Anchors: ", trueP, estimatedP, mse, rmse)
      sumMSE.append(mse)
      sumRMSE.append(rmse)
  After_Total_sumMSE.append(sum(sumMSE))
  After_Each_sumMSE.append(sumMSE)
  After_Total_sumRMSE.append(sum(sumRMSE))
  After_Each_sumRMSE.append(sumRMSE)

avg_sumMSE = np.mean(After_Total_sumMSE)/env.N
avg_sumRMSE = np.mean(After_Total_sumRMSE)/env.N
print("Average MSE: {}, RMSE: {}.".format(avg_sumMSE, avg_sumRMSE))

try:
  os.makedirs(rootpath+'/datas/localizationResult')
except OSError as error:
  print(error)


# np.save(rootpath+'/datas/localizationResult/after_localization_MSE{}_{}_{}'.format(div, pick, model), avg_sumMSE)
# np.save(rootpath+'/datas/localizationResult/after_localization_avgRMSE{}_{}_{}'.format(div, pick, model), avg_sumRMSE)
f = open(rootpath+'/datas/localizationResult/1213_after_localization_avgRMSE{}_{}_{}_v2.pkl'.format(div, pick, model), "wb")
pickle.dump(avg_sumRMSE, f)
f.close()

# np.save(rootpath+'/datas/localizationResult/after_localization_MSE{}_{}_{}'.format(div, pick, model), After_Each_sumMSE)
# np.save(rootpath+'/datas/localizationResult/after_localization_RMSE{}_{}_{}'.format(div, pick, model), After_Each_sumRMSE)

# f = open(rootpath+'/datas/localizationResult/after_localization_MSE{}_{}_{}.pkl'.format(div, pick, model), "wb")
# pickle.dump(After_Each_sumMSE, f)
# f.close()

f = open(rootpath+'/datas/localizationResult/1213_after_localization_RMSE{}_{}_{}_v2.pkl'.format(div, pick, model), "wb")
pickle.dump(After_Each_sumRMSE, f)
f.close()

Average MSE: 112197.44447193027, RMSE: 242.51392321379794.
[Errno 17] File exists: '/content/drive/Shareddrives/KW phase1 repo/datas/localizationResult'


In [695]:
# # After_Each_sumRMSE = np.load(rootpath+'/datas/localizationResult/after_localization_RMSE{}_{}_{}.npy'.format(div, pick, model), allow_pickle=True)
# print(After_Each_sumRMSE)
# from functools import reduce
# from operator import add
# After_sumRMSE = reduce(add ,After_Each_sumRMSE)
# # print(np.array(After_Each_sumMSE).reshape(-1))
# # After_sumMSE = np.concatenate(([], np.array(After_Each_sumMSE).flatten().tolist()),axis=0)
# # After_sumRMSE = [np.log(y) for y in After_sumRMSE]
# print(After_sumRMSE)
# After_RMSEHist, After_bin_edges = np.histogram(After_sumRMSE, bins=50000, range=(0, 1.6*1e3), density=True)
# normalizeAfter_RMSEHist = After_RMSEHist*np.diff(After_bin_edges)

# After_cdf=np.cumsum(normalizeAfter_RMSEHist)

# import matplotlib.pyplot as plt
# fig, ax1 = plt.subplots(1,1, figsize = (8,8))
# ax1.plot(After_bin_edges[:-1], After_cdf, label = 'Localization Error cdf', linewidth=3.0)
# # ax1.hist(After_sumMSE, bins = 10000, label = 'Localization Error cdf', alpha = 0.5, range=(0, 0.06*1e6), cumulative=True, density=True)
# ax1.legend(loc='lower right')
# ax1.set_xlabel('Localization Error')

In [696]:
# fig, ax1 = plt.subplots(1,1, figsize = (8,8))
# plt.grid()
# ax1.ticklabel_format(style='sci', scilimits=(-1,2), axis='x')
# ax1.plot(Before_bin_edges[:-1], Before_cdf, label = 'Before Anomaly Detection', linewidth=3.0)
# ax1.plot(After_bin_edges[:-1], After_cdf, label = 'After Anomaly Detection', linewidth=3.0)
# # ax1.hist(Before_sumMSE, bins = 1000, label = 'Localization Error cdf', range=(0, 1.0*1e6), alpha = 0.5, cumulative=True, density=True)
# ax1.legend(loc='lower right', prop={'size': 12})
# ax1.set_xlabel('RMSE[m]', fontsize=16)
# ax1.set_ylabel('CDF', fontsize=17)
# # plt.savefig("Log-RMSE.svg")
# # plt.savefig(rootpath+'/datas/localizationResult/Log-RMSE{}_{}_{}.svg'.format(div, pick, model))